In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import nltk
from urllib.request import Request, urlopen
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import syllables
import csv

def imports():
    file_data=pd.read_csv('Input.csv')
    return file_data

def scrapper(file_data,i):
    url=file_data['URL'][i]
    print(url)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
    rawpage = requests.get(url,headers=headers)
    plain_text=rawpage.text
    soup=BeautifulSoup(plain_text)
    content = soup.findAll('div',{'class':'td-post-content'})
   
    for i in content:
        value=i.text
    head = soup.findAll('h1',{'class':'entry-title'})
    for i in head:
        content_head=i.text
        
    file_name=url[33:-1]
    with open(("C:\\Users\\JARVIS\\Desktop\\NLP\\WebText\\"+file_name+".txt"),"w",encoding="utf-8") as file:
        file.write(content_head+"\n")
        file.write(value)
    return content_head+value

def process_script(data):
    stop_words = set(stopwords.words('english'))
   
    

    word_tokens = word_tokenize(data)
    filtered_script = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_script = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_script.append(w)
    #print(filtered_script)
    #print(filtered_script)
    #df = pd.DataFrame(filtered_script)
    #print(df)
    return filtered_script
    
def dictionary_creation(data,file_data,x):
    df=pd.read_csv('word_data.csv')
    #print(df.info())
    positive_words=df[df.Positive>0]
    positive_words=list(positive_words['Word'].str.upper())
    negative_words=df[df.Negative>0]
    negative_words=list(negative_words['Word'].str.upper())
 
    positive_count=0
    negative_count=0
    sentences=0
    for i in data:
        if i == '.':
            sentences=sentences+1
        
        if i in positive_words:
            positive_count=positive_count+1
        if i in negative_words:
            negative_count=negative_count+1
    #print("Positive Score-"+str(positive_count))
    #print("negativescore-"+str(negative_count))
    #print("Sentences-"+str(sentences))
    Polarity_Score = (positive_count - negative_count)/ ((positive_count + negative_count) + 0.000001)
    #print("Polarity-Score-"+str(Polarity_Score))
    
    
    personal_pronouns=['I','WE','MY','OURS','US']
    personal_pronoun_count=0
    for i in data:
        if i in personal_pronouns:
            personal_pronoun_count=personal_pronoun_count+1
    #print("Personal Pronoun Count-"+str(personal_pronoun_count))
    
    
    
    complex_words_count=0
    for i in data:
        ss=syllables.estimate(i.lower())
        if ss >2:
            complex_words_count=complex_words_count+1
    #print("Complex Word Count -"+str(complex_words_count))
    
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for i in data:
        if i in punctuations:
            data = list(map(lambda x: x.replace(i,''), data))
    data = [i for i in data if i]
    
    data_len=len(data)
    #print("No. of words- "+str(data_len))
    
    Average_sentence_length=sentences/data_len
    
    Average_Number_of_Words_Per_Sentence=data_len/sentences
    #print('Average_Number_of_Words_Per_Sentence-'+str(Average_Number_of_Words_Per_Sentence))
    
    Percentage_of_Complex_words=complex_words_count/data_len
    #print("Percentage_of_Complex_words-"+str(Percentage_of_Complex_words))
    
    Fog_Index = 0.4 * (Average_Number_of_Words_Per_Sentence + Percentage_of_Complex_words)
    #print("Fog Index-"+str(Fog_Index))
    
    Subjectivity_Score = (positive_count + negative_count)/((data_len) + 0.000001)
    #print("Subjectivity_Score-"+str( Subjectivity_Score))
    
    
    Syllable_count=0
    for i in data:
        ss=syllables.estimate(i.lower())
        Syllable_count=Syllable_count+ss
    #print("Syllable_count-"+str(Syllable_count))
    
    
    
    average_word_length=0
    word_length=0
    for i in data:
        word_length=len(i)+word_length
        
    average_word_length=word_length/data_len
    #print('average_word_length-'+str(average_word_length))
    
    
    file = open('out1.csv', 'a+', newline ='')
    
    with file:
    # identifying header  
        header = ['URL_ID','URL','POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE','SUBJECTIVITY SCORE',
                  'AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE',
                  'COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']
        writer = csv.DictWriter(file, fieldnames = header)
        if x==0:
            writer.writeheader()
        writer.writerow({'URL_ID' : x,
                         'URL':file_data['URL'][x],
                         'POSITIVE SCORE': positive_count,
                         'NEGATIVE SCORE': negative_count, 
                         'POLARITY SCORE': Polarity_Score,
                         'SUBJECTIVITY SCORE':Subjectivity_Score,
                         'AVG SENTENCE LENGTH':Average_sentence_length,
                         'PERCENTAGE OF COMPLEX WORDS':Percentage_of_Complex_words,
                         'FOG INDEX':Fog_Index,
                         'AVG NUMBER OF WORDS PER SENTENCE':Average_Number_of_Words_Per_Sentence,
                         'COMPLEX WORD COUNT':complex_words_count,
                         'WORD COUNT':data_len,
                         'SYLLABLE PER WORD':Syllable_count,
                         'PERSONAL PRONOUNS':personal_pronoun_count,
                         'AVG WORD LENGTH':average_word_length
                         
                         })
        print("SUCESS-"+str(x))
    


    
file_data=imports()
for i in range(len(file_data)):
    data=scrapper(file_data,i)
    data=data.upper()
    df=process_script(data)
    dictionary_creation(df,file_data,i)



https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/
SUCESS-0
https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/
SUCESS-1
https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/
SUCESS-2
https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-successfully-treated-in-healthcare/
SUCESS-3
https://insights.blackcoffer.com/how-artificial-intelligence-can-boost-your-productivity-level/
SUCESS-4
https://insights.blackcoffer.com/how-are-genetic-sequencing-maps-affected-by-deep-learning-and-ai/
SUCESS-5
https://insights.blackcoffer.com/how-is-ai-used-to-solve-traffic-management/
SUCESS-6
https://insights.blackcoffer.com/benefits-of-big-data-in-different-fields/
SUCESS-7
https://insights.blackcoffer.com/how-big-data-will-impact-the-future-of-business/
SUCESS-8
https://insights.blackcoffer.com/how-will-ai-make-decisions-in-tomorrows-wars/
SUCESS-9
https://

SUCESS-82
https://insights.blackcoffer.com/what-is-the-repercussion-of-the-environment-due-to-the-covid-19-pandemic-situation-2/
SUCESS-83
https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-office-space-and-co-working-industries/
SUCESS-84
https://insights.blackcoffer.com/contribution-of-handicrafts-visual-arts-literature-in-the-indian-economy/
SUCESS-85
https://insights.blackcoffer.com/how-covid-19-is-impacting-payment-preferences/
SUCESS-86
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/
SUCESS-87
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis/
SUCESS-88
https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding/
SUCESS-89
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-2/
SUCESS-90
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-3/
SUCESS-91
https://insights.blackcoffer.com/coronavirus-impact-on-the-ho